In [ ]:
from custom_env import CustomEnv
from config import Config

cfg = Config()
env = CustomEnv(cfg)
env.reset()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

MyoSuite:> Registering Myo Envs


In [2]:
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())  # take a random action

In [5]:
info['obs_dict']['touching_info'][0]

0

In [ ]:
from metrics.auc import compute_auc, load_tb_scalar, smooth_values

In [4]:
logdir = "./logs/exp8/RecurrentPPO_1"
tag = "rollout/ep_rew_mean"

steps, values = load_tb_scalar(logdir, tag)

# Apply smoothing
values_smooth = smooth_values(values, window=10)

# Compute AUC
auc = compute_auc(steps, values_smooth)

print(f"AUC for tag '{tag}': {auc:.2f}")

AUC for tag 'rollout/ep_rew_mean': 3094297182.41
